# Testing the `models` module

**Authorship:**
Adam Klie, *10/04/2022*
***
**Description:**
Notebook for testing out the `models` module.

In [1]:
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

import os
import torch
import numpy as np
import pandas as pd
import eugene as eu

Global seed set to 13


GPU is available: True
Number of GPUs: 1
Current GPU: 0
GPUs: Quadro RTX 5000


# BaseModel

# Base Models

# SOTA Models

# Custom Models